# Running BLAST on Spark Cluster

Note that you shall install blast on all nodes for this to run.

In [ ]:
# imports
from pyspark import SparkContext, SparkConf
from chemblhelper import ChEMBLHelper
import pprint
import re
import os
import subprocess
import shutil
import shlex

In [2]:
# initialize Spark
if not 'sc' in globals():
    conf = SparkConf().setMaster('yarn')
    sc = SparkContext(conf=conf)
    
sc.setLogLevel("INFO")

In [25]:
# assume that we already have our proteins.fasta databank created
# see 720 - Similarity Protein Matrix

localProteinsTsv = "proteins.tsv"
hdfsProteinsTsv = "/user/hduser/prosim/proteins.tsv"
blastScript = "/home/hduser/proteins/702blast.sh"

In [26]:
# make sure that /home/hduser/proteins.fasta is available on all hadoop nodes

In [27]:
rawdata = sc.textFile(hdfsProteinsTsv) \
            .map(lambda line: line.split("\t"))

In [70]:
querydata = rawdata.filter(lambda line: (float(line[0])) < 10.0).map(lambda line: str(">ebl|" + line[1] + "|\r\n" + line[2]))

In [67]:
querydata.take(1)

['>ebl|P02708|\r\nMEPWPLLLLFSLCSAGLVLGSEHETRLVAKLFKDYSSVVRPVEDHRQVVEVTVGLQLIQLINVDEVNQIVTTNVRLKQGDMVDLPRPSCVTLGVPLFSHLQNEQWVDYNLKWNPDDYGGVKKIHIPSEKIWRPDLVLYNNADGDFAIVKFTKVLLQYTGHITWTPPAIFKSYCEIIVTHFPFDEQNCSMKLGTWTYDGSVVAINPESDQPDLSNFMESGEWVIKESRGWKHSVTYSCCPDTPYLDITYHFVMQRLPLYFIVNVIIPCLLFSFLTGLVFYLPTDSGEKMTLSISVLLSLTVFLLVIVELIPSTSSAVPLIGKYMLFTMVFVIASIIITVIVINTHHRSPSTHVMPNWVRKVFIDTIPNIMFFSTMKRPSREKQDKKIFTEDIDISDISGKPGPPPMGFHSPLIKHPEVKSAIEGIKYIAETMKSDQESNNAAAEWKYVAMVMDHILLGVFMLVCIIGTLAVFAGRLIELNQQG']

In [58]:
querydata.count()

4

In [71]:
blastedSeqs = querydata.pipe(blastScript)

In [75]:
import datetime
print datetime.datetime.now()

2017-02-11 21:50:04.086305


In [73]:
blastedSeqs.collect()

[u'ebl|P02708|,gnl|ebl|P02708,100.00,482,0,0,1,482,1,482,0.0,992',
 u'ebl|P02708|,gnl|ebl|P32297,45.55,494,198,6,20,471,31,495,1e-146,431',
 u'ebl|P02708|,gnl|ebl|Q15825,43.42,486,221,7,19,476,29,488,4e-137,406',
 u'ebl|P02708|,gnl|ebl|Q15822,43.58,491,211,5,21,471,56,520,3e-128,385',
 u'ebl|P02708|,gnl|ebl|P30532,44.03,452,204,8,21,471,44,447,2e-124,373',
 u'ebl|P02708|,gnl|ebl|P43681,51.44,383,157,3,24,406,37,390,2e-118,362',
 u'ebl|P02708|,gnl|ebl|P43681,42.25,71,36,2,401,471,553,618,1e-11,64.3',
 u'ebl|P02708|,gnl|ebl|Q05901,41.67,468,237,7,11,475,16,450,2e-118,357',
 u'ebl|P02708|,gnl|ebl|P30926,38.58,508,244,8,6,477,9,484,2e-114,348',
 u'ebl|P02708|,gnl|ebl|P17787,38.10,496,245,6,13,477,20,484,5e-111,339',
 u'ebl|P02708|,gnl|ebl|P36544,34.06,502,244,10,19,468,21,487,3e-96,301',
 u'ebl|P02708|,gnl|ebl|Q9UGM1,34.56,489,263,9,11,472,18,476,1e-94,296',
 u'ebl|P02708|,gnl|ebl|Q9GZZ6,33.19,452,267,6,24,471,26,446,1e-93,292',
 u'ebl|P02708|,gnl|ebl|Q07001,33.94,498,254,9,20,471,21,489,7

In [ ]:
print datetime.datetime.now()